# AlexNet with `tf.estimator`

Not interested in `Keras`? Kick off the training wheels and write it in raw `TensorFlow`, you rebel.

In [ ]:
import tensorflow as tf

## Step 2: Import data

In [ ]:
import tflearn.datasets.oxflower17 as oxflower17
x, y = oxflower17.load_data()

In [ ]:
x.shape

## Make an input function

If you're going to use `TensorFlow`, you should get the hang of `tf.data`. It's a little finicky but far easier to use than the old queueing system. And it's incredibly flexible.

*Note- the method below for building a dataset from numpy arrays is straightforward, but will run into memory errors for larger datasets. In those cases you'll need to use a more complicated input pipeline.*

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((x,y))
dataset = dataset.shuffle(500)
dataset = dataset.repeat(100)
dataset = dataset.batch(128)

In [ ]:
def input_fn():
    return dataset.make_one_shot_iterator().get_next()

In [ ]:
input_fn()

## Define a model function

Follow the specification here: https://www.tensorflow.org/guide/custom_estimators


In [ ]:
conv_layers = [(96, 11, 4), 
               "maxpool", 
             (256, 5, 1),
              "maxpool",
              (384, 3, 1),
              (384, 3, 1),
              (256, 3, 1),
              "maxpool"
             ]

In [ ]:
def model_fn(features, labels, mode, params):
    
    is_training = mode == tf.estimator.ModeKeys.TRAIN
    
    print(features.get_shape())
    net = tf.identity(features)
    
    # CONVOLUTIONAL LAYERS
    for l in conv_layers:
        if l == "maxpool":
            net = tf.layers.max_pooling2d(net, 3, 2, padding="valid")
        else:
            f, w, s = l
            net = tf.layers.conv2d(net, f, w, strides=(s,s),
                                  padding="valid", 
                                  activation=tf.nn.relu)
            net = tf.layers.batch_normalization(net)
            
    # DENSE LAYERS
    net = tf.layers.flatten(net)
    for _ in range(2):
        net = tf.layers.dense(net, 2048, activation=tf.nn.relu)
        net = tf.layers.dropout(net, 0.5, training=is_training)
    
    # OUTPUT LAYER
    logits = tf.layers.dense(net, 17)
    probs = tf.nn.softmax(logits)
    predicted_classes = tf.argmax(logits, 1)
    # PREDICT MODE
    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {
            "probabilities":probs,
            "class_ids":predicted_classes
        }
        return tf.estimator.EstimatorSpec(mode, 
                            predictions=predictions)
    # EVALUATE MODE
    labels_oh = tf.one_hot(labels, 17)
    loss = tf.losses.softmax_cross_entropy(labels_oh, logits)
    
    accuracy = tf.metrics.accuracy(labels=labels, 
                                  predictions=predicted_classes)
    metrics = {"accuracy":accuracy}
    tf.summary.scalar("accuracy", accuracy[1])
    
    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(
            mode, loss=loss, eval_metric_ops=metrics
        )
    # TRAIN MODE
    optimizer = tf.train.AdamOptimizer(1e-3)
    train_op = optimizer.minimize(loss, 
                    global_step=tf.train.get_global_step())
    
    return tf.estimator.EstimatorSpec(mode, loss=loss, 
                                    train_op=train_op)
    
    
    

In [ ]:
model = tf.estimator.Estimator(
    model_fn=model_fn, params={}
)

## train it

In [ ]:
model.train(input_fn)